In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function

In [2]:
class L40(object):
    '''Lorenz 40 model of zonal atmospheric flow'''
    
    def __init__(self, members=1, n=40, dt=0.05, F=8):
        self.n = n
        self.dt = dt
        self.dtx = dt
        self.x = np.random.normal(0., 0.1, size=(members, n))
        self.members = members
        self.F = F
    
    def dxdt(self):
        dxdt = np.zeros((self.members, self.n),'f8')
        for n in range(2,self.n-1):
            dxdt[:,n] = -self.x[:,n-2]*self.x[:,n-1] +  \
                        self.x[:,n-1]*self.x[:,n+1] - self.x[:,n] + self.F
        dxdt[:,0] = -self.x[:,self.n-2]*self.x[:,self.n-1] +  \
                self.x[:,self.n-1]*self.x[:,1] - self.x[:,0] + self.F
        dxdt[:,1] = -self.x[:,self.n-1]*self.x[:,0] + \
                self.x[:,0]*self.x[:,2] - self.x[:,1] + self.F
        dxdt[:,self.n-1] = -self.x[:,self.n-3]*self.x[:,self.n-2] + \
                            self.x[:,self.n-2]*self.x[:,0] - \
                            self.x[:,self.n-1] + self.F
        return dxdt
    
    def rk4step(self):
        h = self.dt; hh = 0.5*h; h6 = h/6.
        x = self.x
        dxdt1 = self.dxdt()
        self.x = x + hh*dxdt1
        dxdt2 = self.dxdt()
        self.x = x + hh*dxdt2
        dxdt = self.dxdt()
        self.x = x + h*dxdt
        dxdt2 = 2.0*(dxdt2 + dxdt)
        dxdt = self.dxdt()
        self.x = x + h6*(dxdt1 + dxdt + dxdt2)

        


In [84]:
members = 50

truth = L40(members=1)
for n in range(200):  # Initialize truth with a mature state
    truth.rk4step()

ensemble = L40(members=members)
for n in range(members):
    ensemble.x[n] = truth.x + 0.1*np.random.randn(40)
    


In [85]:
for n in range(40):
    truth.rk4step()
    ensemble.rk4step()

xbar = ensemble.x.mean(axis=0)
xprime = ensemble.x - xbar
Pb = np.sum(xprime[:, np.newaxis, :]*xprime[:, :, np.newaxis], axis=0)/(members-1)

In [86]:
std_obs = 0.5
var_obs = std_obs**2
obs = truth.x[0] + std_obs*np.random.randn(40)
R = var_obs * np.eye(40)

In [88]:
K = np.dot(Pb, np.linalg.inv(Pb+R))

In [106]:
xbar_a = xbar + np.dot(K, obs-xbar)

xprime_a = np.zeros_like(xprime)
for n in range(members):
    obs_prime = std_obs*np.random.randn(40)
    xprime_a[n] = xprime[n] + np.dot(K, obs_prime-xprime[n])

ensemble.x = xbar_a + xprime_a